# Waveform h5 Files test

* Aim: This script is desinged to test the waveforms h5 files.

* Author: Bhavesh Khamesra

* Date:22nd January 2019

* Tests to be added:

  * Check metadata entries - name and value type
  * Basic parameters checks - spin dimension <1, simulation-type based on spins, m1+m2 = 1, norm(nhat)=1, norm(Lhat)=1, 
  
* Important Remarks:
  * Currently this script has been written explicitly to test the Georgia Tech waveforms in their current state and using the current infrastructure. It is definitely possible to extend it to test any other waveforms but cannot be used in its present state directly. Please follow the comments in the script to see where the changes are required
  * Currently only checks for BBH - update for other systems
  

In [1]:
import numpy as np
import lal, pycbc
import lalsimulation as lalsim
import pycbc.types, pycbc.filter
import h5py
from lal import MSUN_SI, PC_SI, PI, C_SI, G_SI
%matplotlib inline
import matplotlib
import matplotlib.pyplot as P

import sys
sys.path.append('/home/bhavesh.khamesra/Codes')

import romspline, os, glob, h5py
import shutil as sh

### Extracting waveform metadata from h5 file using romspline

In [2]:
msun = MSUN_SI

In [3]:
def simulation_type(spin1, spin2):

    spin_bh1 = np.around(spin1, decimals=5)
    spin_bh2 = np.around(spin2, decimals=5)
    
    if (np.count_nonzero(spin_bh1) ==0 and np.count_nonzero(spin_bh2)==0):
        simtype = 'non-spinning'
    elif (np.count_nonzero(spin_bh2[0:2])>0 or np.count_nonzero(spin_bh2[0:2])>0) :
        simtype = 'precessing'
    else:
        simtype = 'aligned-spins'

    return simtype


# Waveform Checks

## Metadata

### Checks added - 
* masses, eta - individual masses should be less 1, sum should be close to 1, eta = m1*m2/m_total**2
* spins - mag of spins should be less than 1
* fixed entries - 'Format', 'type', 'NR-group', 'NR-code', 'point-of-contact-email', 'INSPIRE-bibtex-keys', 'license','NR-techniques', 'PN_approximant'
* Variable Entries - 'name', 'alternative-names', 'modification-data',  'simulation-type', 'production-run', 'object1', 'object2', 'LNhatx', 'LNhaty', 'LNhatz', 'nhatx', 'nhaty', 'nhatz'
* production-run - set to 0 if D<7, or resolution <M/120 - done
* lmax - check from amp and phase key values


### Remaining Entries to check - 

* files-in-error-series,
* comparable-simulations, 
* f_lower_at_1MSUN - Extract the phase values and compute this quantity, should match with one in metadata. If disagreement then reject the waveform/raise Error  
* Omega
* eccentricity
* mean_anomaly

### Additional Things
* Think additional checks for spins and masses


## Data:
### Checks added - 
* Check if all modes are present for all |m|<l, l from (2 ... lmax)
* Check if m=0 modes are set to zero

### Remaining checks to add - 

* Check if data is mean centered
* Create a check for extrapolation of waveforms. 
* Check if junk radiation is removed
* Check if end of each mode goes smoothly to zero
* Check if m, -m modes for non-precessing cases have overlap of 1
* Check if (2,2) mode is dominant mode
* Check for modes which are noise dominant - need to develop a test 


In [4]:


def read_metadata(h5path_new):
    """Read the metadata from h5file 
    """
    #h5file = h5py.File(h5path_new)
    with h5py.File(h5path_new) as h5file:
        metadata = {}

        for item in h5file.attrs.keys():
            metadata[item] = h5file.attrs[item]


        #Extract the amplitude and phase from h5. If a field is missing, send it to failed directory
        amp, phase = {}, {}

        for key in h5file.keys():

            if key[:3] =="amp":
                amp_lm =  h5file[key]
                ampkey = amp_lm.keys()

                #For missing data for some mode in amplitude - reject the waveform
                if len(ampkey)==0:
                    raise ValueError("Data from amplitude - %s missing"%ampkey)


                t_val   = amp_lm[ampkey[0]][:]
                amp_val = amp_lm[ampkey[1]][:]
                deg_val = amp_lm[ampkey[2]].value
                err_val = amp_lm[ampkey[3]][:]
                tol_val = amp_lm[ampkey[4]].value

                amp[key] = {"X":t_val, "Y":amp_val, "deg":deg_val, "tol":tol_val, "errors":err_val}

            elif key[:5] =="phase":
                ph_lm = h5file[key]
                phkey = ph_lm.keys()

                #For missing data for some mode in phase  - reject the waveform
                if len(phkey)==0:
                    raise ValueError("Data from phase - %s missing"%phkey)


                t_val   = ph_lm[phkey[0]][:]
                ph_val  = ph_lm[phkey[1]][:]
                deg_val = ph_lm[phkey[2]].value
                err_val = ph_lm[phkey[3]][:]
                tol_val = ph_lm[phkey[4]].value

                phase[key] = {"X":t_val, "Y":ph_val, "deg":deg_val, "tol":tol_val, "errors":err_val}

        #h5_file.close()
        return [metadata, amp, phase]
    
    
def check_mass(metadata): 
    """Check mass parameters
    """
    
    m1 = metadata['mass1']
    m2 = metadata['mass2']
    
    if m1>1: raise ValueError('Mass of BH1 cannot be greater than 1 as total mass of system is normalized')
    
    if m2>1: raise ValueError('Mass of BH2 cannot be greater than 1 as total mass of system is normalized')
        
    #if not(np.around(m1+m2, decimals=2)==1): raise ValueError('Sum of masses is not close to 1. Please check the mass entries.')
    
    eta = m1*m2/(m1+m2)**2. 
    assert(np.isclose(eta,metadata['eta'])), 'Symmetric Mass Ratio does not match mass values. Please check the value of eta.'
   
    return


def check_names(metadata):
    """check the name entries
    """
    
    wfdata = np.genfromtxt('../wf_junkrad.txt', dtype=None, comments='#', usecols=(0,1), skip_header=1, delimiter = '\t', names = ('GTID', 'simname'))
    GTname, wfname = wfdata['GTID'], wfdata['simname']

    simname = metadata['name']
    idx = np.where(simname==GTname)
    if len(idx)<1:
        raise ValueError('Simulation not found in wf_junkrad.txt. Either check the simulation tag or update the wf_junkrad.txt file')
    elif len(idx)>1:
        raise ValueError('Multiple Entries found in wf_junkrad.txt file. Please check and update the file.')
    else:
        assert (metadata['alternative-names']==wfname[idx]), 'Alternative name does not match with simulation name in database. Please check the entry.'
        
    return
    


def check_spin(metadata):
    """Check Spin parameters
    """
    
    a1 = np.array((metadata['spin1x'], metadata['spin1y'], metadata['spin1z']))
    a2 = np.array((metadata['spin2x'], metadata['spin2y'], metadata['spin2z']))
    
    assert(np.linalg.norm(a1)<=1 ), 'Dimensionless Spin of BH1 cannot be greater than 1. Please check the spin values'
    assert(np.linalg.norm(a2)<=1 ), 'Dimensionless Spin of BH2 cannot be greater than 1. Please check the spin values'
    
    simtype = simulation_type(a1, a2)
    
    #Need to fix this - Spin type originally determined by spins provided at t=0 but due to junk radiation, there can 
    #be some non-zero spins which can lead to different spin-type
    
    #assert(simtype==metadata['simulation-type']), 'Simulation type {} does not match entry from metadata - {}! Please check the entry.'.format(simtype, metadata['simulation-type'])

    return 


#Comment - should modify this based on number of gravitational wave cycles!
def check_productionruns(h5file_path):
    """Check if this run is production quality"""
    
    #Extract the separation from auxiliary info
    #h5_file = h5py.File(h5file_path)
    with h5py.File(h5file_path) as h5_file:
        aux_data = h5_file['auxiliary-info']
        separation = aux_data.attrs['separation']

        #Extract resolution from simulation name (assuming all simulaiton names have this info - checked with catalog)
        simname = h5_file.attrs['alternative-names']
        res_in_name = 1
        if 'M' in simname:
            res = (simname.split('M'))[1]
            res = res.split('_')[0]
            res = int(res)
        elif 'm' in simname:
            res = (simname.split('m'))[1]
            res = res.split('_')[0]
            res = int(res)
        elif 'res' in simname:
            res = (simname.split('M'))[1]
            res = res.split('_')[0]
            res = int(res)
        else:
            print('Resolution not found in simulation name')
            res_in_name=0

        prodrun = 1
        if separation<7: 
            prodrun = 0

        if res_in_name==1:
            if res<120.: prodrun = 0


        if not(prodrun==h5_file.attrs['production-run']):
            raise ValueError('This run in not production quality as either resolution is less than M/120 or separation is smaller than 7M. Please check \
            the production run tag again in metadata.')

        #h5_file.close()
        return
        
    
def check_lmax(h5file_path):
    """check for the lmax value and if all the modes have been provided"""
    
    #Extract the lmax value
    #h5_file = h5py.File(h5file_path)
    with h5py.File(h5file_path) as h5_file:
        lmax_metadata = h5_file.attrs['Lmax']


        lmax=0
        for key in h5_file.keys():

            if key[:3]=='amp':
                l = int((key.split('l')[1]).split('_')[0])
                if l>lmax:lmax=l
            elif key[:5]=='phase':
                l = int((key.split('l')[1]).split('_')[0])
                if l>lmax:lmax=l
            else: continue

        assert(lmax==lmax_metadata), 'Lmax value does not agree between metadata and amplitude/phase l modes present in the data.'
        #h5_file.close()
        return
    
    

def check_metadata(h5file_path):
    """Check the metadata entries from h5file
    """
    
    #h5file = h5py.File(h5file_path)
    with h5py.File(h5file_path) as h5file:
    
        metadata = {}

        for item in h5file.attrs.keys():
            metadata[item] = h5file.attrs[item]

        #h5file.close()
        
    metadata_fixed_entries = ['Format', 'type', 'name', 'alternative-names', 'NR-group', 'NR-code', \
                       'modification-date', 'point-of-contact-email', 'simulation-type', \
                       'INSPIRE-bibtex-keys', 'license', 'Lmax', 'NR-techniques', 'files-in-error-series',\
                       'comparable-simulation', 'production-run', 'object1', 'object2', 'mass1', \
                        'mass2', 'eta','f_lower_at_1MSUN', 'spin1x', 'spin1y', 'spin1z', 'spin2x',\
                        'spin2y', 'spin2z', 'LNhatx', 'LNhaty', 'LNhatz', 'nhatx', 'nhaty', 'nhatz',\
                        'Omega', 'eccentricity', 'mean_anomaly', 'PN_approximant' ]
    
    #Basic check - if all metadata entries are present
    for item in metadata_fixed_entries:
        assert(item in metadata.keys()),"%s entry not found, please check the %s file again"%(item, os.path.basename(h5file_path))
            
            
    nr_technique_extrwf = 'Puncture-ID, BSSN, Psi4-integrated, Extrapolated-Waveform, ApproxKillingVector-Spin, Christodoulou-Mass'
    nr_technique_finrad = 'Puncture-ID, BSSN, Psi4-integrated, Finite-Radius-Waveform, ApproxKillingVector-Spin, Christodoulou-Mass'
    

    
    #Fixed Entries check - Modify this part if testing waveforms outside GT:
    
    assert(metadata['Format']>0), 'Format should be greater than 0.'
    assert(metadata['type']=='NRinjection'), 'Type has to be NRinjection for NR waveform.'
    
    assert(isinstance(metadata['NR-group'], basestring)), 'NR group has to be string'
    
    #For waveform outside GT, comment the below line
    assert(metadata['NR-group']=='Georgia Tech'), 'Waveform outside Georgia Tech'
    
    #For waveform outside GT, add code names
    assert(metadata['NR-code']=='MAYA' or metadata['NR-code']=='Einstein Toolkit' ), 'Code is not Maya'
    assert(isinstance(metadata['point-of-contact-email'], basestring)), 'Contact email missing'
    
    assert(isinstance(metadata['INSPIRE-bibtex-keys'], basestring)), 'bibtex keys for citation missing'
    assert(metadata['license']=='public' or metadata['license']=='LVC-internal' ), 'License key incorrect'
    
    #For waveform outside GT, add additional code techniques from NR Injection infrastructure paper
    assert(metadata['NR-techniques']==nr_technique_extrwf or metadata['NR-techniques']==nr_technique_finrad ),'NR-technique incorrect or modified \
    - {}!'.format(metadata['NR-techniques'])
    assert(isinstance(metadata['PN_approximant'], basestring)), 'Approximant type has to be a string.'
    

    #Variable Entries: masses, spins, nhat, LNhat
    
    #Check names:
    check_names(metadata)
    
   
    #Check for object type:
    assert(metadata['object1']=='BH'), 'Object 1 has to be BH.'
    assert(metadata['object2']=='BH'),'Object 2 has to be BH.'
    
    
    #Check mass and Spin entries
    check_mass(metadata)
    
    check_spin(metadata)
    
    
    #Check for nhat and LNhat
    nhat = np.array((metadata['nhatx'], metadata['nhaty'], metadata['nhatz']))
    LNhat = np.array((metadata['LNhatx'], metadata['LNhaty'], metadata['LNhatz']))
    
    assert(np.isclose(np.linalg.norm(nhat), 1, atol = 1e-8) ), 'Direction (orbital separation unit vector) cannot be greater than 1. Please check the direction values'
    assert(np.isclose(np.linalg.norm(LNhat), 1, atol = 1e-8) ), 'Direction (Angular Momentum unit vector) cannot be greater than 1. Please check the angular momentum values'
    
   
    #Check production run - 
    check_productionruns(h5file_path)
     
   
    #Check lmax - 
    check_lmax(h5file_path)
    
    print('Metadata Checks - Basic Checks on metadata completed.')
    

## Waveform Basic data check:

1. Check if all modes are present
2. Check if all modes have the necessary data 

In [5]:
#Checks if all the modes are present
def check_modespresence(h5file_path):
    """check if all the modes less than lmax are present in amplitude and phase"""
    
    #Extract the lmax value
    #h5_file = h5py.File(h5file_path)
    with h5py.File(h5file_path) as h5_file:
        lmax_metadata = h5_file.attrs['Lmax']

        #Check if all the l and m modes are present for amplitude and phase
        for ll in range(2, lmax_metadata):
            for mm in range(-ll, ll+1):
                amp_key = 'amp_l%d_m%d'%(ll, mm)
                ph_key = 'phase_l%d_m%d'%(ll, mm)

                assert amp_key in h5_file.keys(), '%s key missing'%amp_key
                assert ph_key in h5_file.keys(), '%s key missing'%amp_key

        print('All modes found for the waveform %s'%h5_file.attrs['name'])

        #h5_file.close()
        return


#Checks if each there exists data for each mode for phase and amplitude
def check_modecontent(h5file_path):
    """Only check if some data present in all the modes except m=0. Raise warning is some mode is lacking data"""
    
    #h5_file = h5py.File(h5file_path)
    with h5py.File(h5file_path) as h5_file:
        simname = h5_file.attrs['name']

        #check if each amplitude and phase group has relevant data
        for grp_name in  h5_file.keys():

            if grp_name=='auxiliary-info':continue

            group_lm =  h5_file[grp_name]
            groupkey = group_lm.keys()

            assert(len(groupkey)==5), '%s: Group %s missing some keys'%(simname, grp_name)

            m = int(grp_name.split('_m')[1])

            if m==0:
                var_val  = group_lm[groupkey[1]][:]
                if np.any(var_val)>0: raise ValueError('%s: m=0 Modes have to be set to zero for all modes. Non-zero value encountered for %s'%(simname,grp_name))

            for k in np.array((0,1)): 
                if m==0: continue
                assert(len(group_lm[groupkey[k]])>0),'%s: Group %s is missing data in key %s (length=%f)' %(simname, grp_name, groupkey[k], len(group_lm[groupkey[k]]))


        print('%s:Mode Content checks complete. All m=0 modes have data set to zero while data present for remaining modes.'%simname)
        #h5_file.close()
        return






### Extract the waveform data from h5 using lalsimulation

In [6]:


def set_mode_in_params(params, load_all_modes, lm_array, include_minusm_mode=True):
    """ Sets modes in params dict.
        Only adds (l,m) and (l,-m) modes.
    """

    # First, create the 'empty' mode array
    ma=lalsim.SimInspiralCreateModeArray()

    if load_all_modes:
        lalsim.SimInspiralModeArrayActivateAllModes(ma)    #Need to check this!
    else:
        for modes in lm_array:
            # add (l,m) 
            lalsim.SimInspiralModeArrayActivateMode(ma, modes[0], modes[1])
            if include_minusm_mode:
                #add (l,-m) modes
                lalsim.SimInspiralModeArrayActivateMode(ma, modes[0], -1*modes[1])
        
    #then insert your ModeArray into the LALDict params with
    lalsim.SimInspiralWaveformParamsInsertModeArray(params, ma)

    return params


def load_lvcnr_data(NRh5File, filepath, deltaT=1./4096, load_all_modes=True, lm_array=[], include_minusm_mode=True):
    """ Loads waveform from nrh5 file. 
   load_all_modes - set to true if you want all modes else set to false if you want selected modes. 
   lm_array - add the selected modes. Only select +m modes (l,-m modes get loaded by default). Leave empty if load_all_modes is True 
    """

    # set mode for NR data
    params_NR = lal.CreateDict()
    lalsim.SimInspiralWaveformParamsInsertNumRelData(params_NR, filepath)
    
    params_NR = set_mode_in_params(params_NR, load_all_modes, lm_array, include_minusm_mode=True)

    # Choose extrinsic parameters:
    mtotal = 100


    # Metadata parameters masses:
    m1 = NRh5File.attrs['mass1']
    m2 = NRh5File.attrs['mass2']
    m1SI = m1 * mtotal/(m1 + m2) * MSUN_SI
    m2SI = m2 * mtotal/(m1 + m2) * MSUN_SI

    # This will use the entire data
    f_lower = 0

    inclination = 0
    distance = 100. * PC_SI * 1.0e6
    phiRef = 0.0 #np.pi/2.

    fRef = f_lower
    spins = lalsim.SimInspiralNRWaveformGetSpinsFromHDF5File(fRef, mtotal, filepath)
   
    s1x = spins[0]
    s1y = spins[1]
    s1z = spins[2]
    s2x = spins[3]
    s2y = spins[4]
    s2z = spins[5]

    
    # If f_lower == 0, update it to the start frequency so that SEOBNRv4 gets the right start
    # frequency
    if f_lower == 0:
        f_lower = NRh5File.attrs['f_lower_at_1MSUN']/mtotal
        #print('Frequency from metadata of NR waveforms  = %g'%f_lower)
    fRef = f_lower
    fStart = fRef

    #fRef = -1.
    # Generating the NR waveform
    approx = lalsim.NR_hdf5
    hp, hc = lalsim.SimInspiralChooseTDWaveform(m1SI, m2SI, s1x, s1y, s1z,
               s2x, s2y, s2z, distance, inclination, phiRef, 0.0, 0.0, 0.0,
               deltaT, fStart, fRef, params_NR, approx)
    times = np.arange(len(hp.data.data))*hp.deltaT

    return times, hp, hc, mtotal, m1SI, m2SI, s1x, s1y, s1z, s2x, s2y, s2z, distance, \
        inclination, phiRef, deltaT, fStart, fRef

def shift_peak_to_zero(t, hp, hc):
    h = hp.data.data -1j * hc.data.data
    peakIdx = np.argmax(np.abs(h))
    t -= t[peakIdx]
    return t, h


## Waveform Data Checks (Continued):

1. Check if data is mean centered - Done
2. Create a check for extrapolation of waveforms.
3. Check if junk radiation is removed
4. Check if end of each mode goes smoothly to zero
5. Check if m, -m modes for non-precessing cases have overlap of 1
6. Check if (2,2) mode is dominant mode
7. Check for modes which are noise dominant - need to develop a test


In [7]:
def check_meancentering(h5filepath):
    
    #NRh5File = h5py.File(h5filepath, 'r')
    with h5py.File(h5filepath, 'r') as NRh5File:
        simname = NRh5File.attrs['name']
        # Generate the lvcnr data with only (2,\pm 2) modes and get the parameters that go into lalsimulation
        t_NR, hp_NR, hc_NR, mtotal, m1SI, m2SI, s1x, s1y, s1z, s2x, s2y, s2z, distance, inclination, \
        phiRef, deltaT, fStart, fRef = load_lvcnr_data(NRh5File, h5filepath, deltaT=1./4096, load_all_modes=True)
        #NRh5File.close()
    
    # shift time array such that peak occurs at t=0
    t_NR_shift, h_NR_shift = shift_peak_to_zero(t_NR, hp_NR, hc_NR)

    time_diff = np.divide((t_NR_shift-t_NR), t_NR)
    
    time_diff = time_diff[np.logical_not(np.isnan(time_diff))]
    
    if np.amax(time_diff)>5:
        raise ValueError('%s: Waveform is not mean centered!'%simname)
    elif np.amax(time_diff)>1:
        print("%s:Warning - Mean centering was not perfectly achieved"%simname)
    else: 
        print("%s: Waveform is mean centered"%simname)
    

def check_modedominancy(h5filepath):
    
    #NRh5File = h5py.File(h5filepath, 'r')
    with h5py.File(h5filepath, 'r') as NRh5File:
        simname = NRh5File.attrs['name']

        lmax = NRh5File.attrs['Lmax']

        lm_modes = []
        for l in range(2,lmax+1):
            m_vals = np.arange(-l, l+1, 1)
            for m in m_vals:
                lm_modes.append((l,m))


        # Generate the lvcnr data with only (2,\pm 2) modes and get the parameters that go into lalsimulation
        t22_NR, hp22_NR, hc22_NR, mtotal, m1SI, m2SI, s1x, s1y, s1z, s2x, s2y, s2z, distance, inclination, \
        phiRef, deltaT, fStart, fRef = load_lvcnr_data(NRh5File, h5filepath, deltaT=1./4096, load_all_modes=False, lm_array=[(2,2)], include_minusm_mode=False)  

        for item in lm_modes:
            t_lm_NR, hp_lm_NR, hc_lm_NR, mtotal, m1SI, m2SI, s1x, s1y, s1z, s2x, s2y, s2z, distance, inclination, \
        phiRef, deltaT, fStart, fRef = load_lvcnr_data(NRh5File, h5filepath, deltaT=1./4096, load_all_modes=False, lm_array=[item], include_minusm_mode=False)

            h_lm = hp_lm_NR.data.data -1j * hc_lm_NR.data.data
            amp_max = np.amax(np.absolute(h_lm))
            amp22_max = np.amax(np.absolute(hp22_NR.data.data -1j * hc22_NR.data.data))

            if amp_max>amp22_max: raise ValueError('For face on case, {} mode found to be dominant! Please check the data.'.format(item))

        print('Mode Checks complete - (2,2) is dominant mode for face-on case.')
        #NRh5File.close()



def compare_initfrequency(h5filepath):
    
    #NRh5File = h5py.File(h5filepath, 'r')
    with h5py.File(h5filepath, 'r') as NRh5File:
    
        # Generate the lvcnr data with only (2,\pm 2) modes and get the parameters that go into lalsimulation

        t_22_NR, hp_22_NR, hc_22_NR, mtotal, m1SI, m2SI, s1x, s1y, s1z, s2x, s2y, s2z, distance, inclination, \
        phiRef, deltaT, fStart, fRef = load_lvcnr_data(NRh5File, h5filepath, deltaT=1./4096, load_all_modes=False, lm_array=[(2,2)], include_minusm_mode=False)
        #NRh5File.close()

    # shift time array such that peak occurs at t=0
    t_22_NR, h_22_NR = shift_peak_to_zero(t_22_NR, hp_22_NR, hc_22_NR)

    # Compute the start frequency and error w.r.t metadata
    phi22_NR = np.unwrap(np.angle(h_22_NR))
    omega22_NR = -1*np.gradient(phi22_NR)/np.gradient(t_22_NR)
    flow_NR = omega22_NR[0]/2/np.pi
    diff_NR = fStart - flow_NR

    assert(diff_NR<0.2), '%s, Metadata frequency = %g does not match with frequency from data = %g.'% (os.path.basename(h5filepath), fStart, flow_NR) #, fStart-flow_NR)
    


In [8]:
def equalize_timeseries(strain1, strain2, t1, t2):

    # Make the length of waveforms equal - We want to keep same time duration for inspiral (backward) and merger-ringdown (forward) part 
    
    wf1_maxidx = np.argmax(np.absolute(strain1))
    wf1_fwd_tlen = len(strain1.real[wf1_maxidx:])   #Inspiral part
    wf1_bwd_tlen = len(strain1.real[:wf1_maxidx])   #Merger-ringdown part

    wf2_maxidx = np.argmax(np.absolute(strain2))
    wf2_fwd_tlen = len(strain2.real[wf2_maxidx:])
    wf2_bwd_tlen = len(strain2.real[:wf2_maxidx])


    if wf1_fwd_tlen>wf2_fwd_tlen:
        wf1_fwd = strain1.real[wf1_maxidx:wf1_maxidx+wf2_fwd_tlen]
        wf2_fwd = strain2.real[wf2_maxidx:wf2_maxidx+wf2_fwd_tlen]
        t_fwd = t2[wf2_maxidx:wf2_maxidx+wf2_fwd_tlen]
    else:
        wf1_fwd = strain1.real[wf1_maxidx:wf1_maxidx+wf1_fwd_tlen]
        wf2_fwd = strain2.real[wf2_maxidx:wf2_maxidx+wf1_fwd_tlen]
        t_fwd = t2[wf2_maxidx:wf2_maxidx+wf1_fwd_tlen]


    if wf1_bwd_tlen>wf2_bwd_tlen:
        diff = wf1_bwd_tlen - wf2_bwd_tlen
        wf1_bwd = strain1.real[diff:wf1_maxidx]
        wf2_bwd = strain2.real[0:wf2_maxidx]
        t_bwd = t2[0:wf2_maxidx]
    else:
        diff = abs(wf1_bwd_tlen - wf2_bwd_tlen)
        wf1_bwd = strain1.real[0:wf1_maxidx]
        wf2_bwd = strain2.real[diff:wf2_maxidx]
        t_bwd = t2[diff:wf2_maxidx]


    wf1 = np.append(wf1_bwd, wf1_fwd)
    wf2 = np.append(wf2_bwd, wf2_fwd)
    time = np.append(t_bwd, t_fwd)

    return wf1, wf2, time

    
def compute_matches(h5filepath):
   
    # Generate NR waveform
    
    #NRh5File = h5py.File(h5filepath, 'r')
    with h5py.File(h5filepath, 'r') as NRh5File:
        simtype = NRh5File.attrs['simulation-type']
        t_NR, hp_NR, hc_NR, mtotal, m1SI, m2SI, s1x, s1y, s1z, s2x, s2y, s2z, distance, inclination, phiRef, deltaT,\
        fStart, fRef = load_lvcnr_data(NRh5File, h5filepath, deltaT=1./4096, load_all_modes=True, lm_array=[], include_minusm_mode=True)
        #NRh5File.close()
    
    #flow_metadata = read_flow(filepath)
    print("Waveform Details: \n")
    print(" m1 = %g Msun, m2 = %g Msun, q = %g\n s1 = (%g, %g, %g) \n s2 = (%g, %g, %g) \n distance=%g, inclination = %g, phiref = %g \n"%(m1SI/msun, m2SI/msun, m1SI/m2SI, s1x, s1y, s1z, s2x, s2y, s2z, distance, inclination, phiRef))

    t_NR, h_NR = shift_peak_to_zero(t_NR, hp_NR, hc_NR)
    
    
    # Generate IMR Phenom Waveform model - to compare aginst lvcnr - works with Precessing also
    waveform_model1 = 'IMRPhenomPv2'

    # Generate the waveform with the same parameters as lvcnr, including the same start frequency
    params_ph = lal.CreateDict()
    approx_ph = lalsim.GetApproximantFromString(waveform_model1)

    #change this for better overlap
    phiRef_ph = 0

    fRef_IMR = -1
    hp_ph, hc_ph = lalsim.SimInspiralChooseTDWaveform(m1SI, m2SI, s1x, s1y, s1z,
                    s2x, s2y, s2z, distance, inclination, phiRef_ph, 0.0, 0.0, 0.0,
                    deltaT, fStart, fRef_IMR, params_ph, approx_ph)
    t_ph = np.arange(len(hp_ph.data.data))*hp_ph.deltaT

    # shift time array such that peak occurs at t=0
    t_ph, h_ph = shift_peak_to_zero(t_ph, hp_ph, hc_ph)
    
    wf_NR, wf_ph, time_ph = equalize_timeseries(h_NR, h_ph, t_NR, t_ph)
    wf_nr_data = pycbc.types.TimeSeries(wf_NR, deltaT)      
    wf_ph_data = pycbc.types.TimeSeries(wf_ph, deltaT)
    match_nr_ph = pycbc.filter.match(wf_nr_data, wf_ph_data)

    print("Match between NR and Phenom = {}".format(match_nr_ph))

    # Generate SEOBNR waveform - does not work with precessing - update this when new model comes out
    
    
    if not(simtype=='precessing'):
        print("Waveform not precessing")
        waveform_model2 = 'SEOBNRv4'
        
        # Generate the waveform with the same parameters as lvcnr, including the same start frequency
        params_seob = lal.CreateDict()
        approx_seob = lalsim.GetApproximantFromString(waveform_model2)

        #change this for better overlap
        phiRef_seob =0. 

        hp_seob, hc_seob = lalsim.SimInspiralChooseTDWaveform(m1SI, m2SI, s1x, s1y, s1z,
                        s2x, s2y, s2z, distance, inclination, phiRef_seob, 0.0, 0.0, 0.0,
                        deltaT, fStart, fRef, params_seob, approx_seob)
        t_seob = np.arange(len(hp_seob.data.data))*hp_seob.deltaT

        # shift time array such that peak occurs at t=0
        t_seob, h_seob = shift_peak_to_zero(t_seob, hp_seob, hc_seob)
                                     

        # Compute the matches
        wf_NR, wf_seob, time_seob = equalize_timeseries(h_NR, h_seob, t_NR, t_seob)

        wf_nr_data = pycbc.types.TimeSeries(wf_NR, deltaT)
        wf_seob_data = pycbc.types.TimeSeries(wf_seob, deltaT)
        match_nr_seob = pycbc.filter.match(wf_nr_data, wf_seob_data)
        
    
        print("Match between NR and SEOBNR = {}".format(match_nr_seob))
        
        # Compute the matches
        wf_ph, wf_seob, time_seob = equalize_timeseries(h_ph, h_seob, t_ph, t_seob)

        wf_ph_data = pycbc.types.TimeSeries(wf_ph, deltaT)
        wf_seob_data = pycbc.types.TimeSeries(wf_seob, deltaT)
        
        match_ph_seob = pycbc.filter.match(wf_ph_data, wf_seob_data)
    
        print("Match between Phenom and SEOBNR = {}".format(match_ph_seob))
                                     
    #Reject the waveform if the match is low
    if (simtype=='precessing'):
        if (match_nr_ph<0.95) or (match_nr_seob>0.95): 
            raise ValueError('High mismatches between NR and Approximate models. Please check the waveform!')
        elif (match_nr_ph<0.99) or (match_nr_seob>0.99): 
            print('Warning: Matches between NR and Approximate modes are lower than expected')
    
    else :
        if (match_nr_ph<0.99) or (match_nr_seob<0.99): 
            raise ValueError('High mismatches between NR and Approximate models. Please check the waveform!')
            #print('Warning: Matches between NR and Approximate modes are lower than expected')
       
    

In [10]:

import os, glob
import shutil as sh

#wfname = ['GT0888', 'GT0905']#, 'GT0760', 'GT0757', 'GT0748','GT0717','GT0653', 'GT0652', 'GT0651', 'GT0622','GT0617']
#lvcnr_file = sorted(glob.glob('/home/bhavesh.khamesra/Data/TestWaveforms/H5Files/*/%s.h5'%wfname[0]))
#lvcnr_file = sorted(glob.glob('/home/deborah.ferguson/H5Files/NonSpinning/GT0615.h5'))
lvcnr_file = glob.glob('/home/bhavesh.khamesra/Projects/BBH_Injections/IMBH_Studies/GT0577.h5')

print lvcnr_file
test_dir = '/home/bhavesh.khamesra/Data/TestDirectory'
if not os.path.exists(test_dir):
    os.makedirs(test_dir)

    
success_test_dir = test_dir+'/Succeeded'
if not os.path.exists(success_test_dir):
    os.makedirs(success_test_dir)

fail_test_dir = test_dir+'/Failed'
if not os.path.exists(fail_test_dir):
    os.makedirs(fail_test_dir)


succeeded = 0
failed = 0

i=0
for wf_file in lvcnr_file:
    print("\n\n%d. Waveform Checking - %s \n"%(i,os.path.basename(wf_file)))
    i+=1
    wf_name = os.path.basename(wf_file)
    wf_path = wf_file
    #try:
        ##Check for metadata
    check_metadata(wf_path)
    print("passed check_metadata")
        
     ##Check if all the modes are present
    check_modespresence(wf_path)
    print("passed check_modespresence")

    #Check if any information is missing for any mode
    check_modecontent(wf_path)
    print("passed check_modecontent")

    check_meancentering(wf_path)     #Check this
    print("passed check_meancentering")
        
    check_modedominancy(wf_path)     #Check this
    print("passed check_modedominancy")
    
    compare_initfrequency(wf_path)  
    print("passed compare_initfrequency")
        
    compute_matches(wf_path)     #Check this
    print("passed compute_matches")
        
    wf_path = os.path.join(success_test_dir, wf_name)
    sh.copyfile(wf_file, wf_path)
    print('\033[92m' + wf_file+" passed all tests \033[0m")
    succeeded+=1
        
   # except Exception as e:
   #     print('\033[91m' + wf_file+" failed with exception \n"+str(e) + '\033[0m')
        
   #     wf_path = os.path.join(fail_test_dir, wf_name)
   #     sh.copyfile(wf_file, wf_path)
        
    #    failed += 1
#print("%s runs passed the test" %succeeded)
#print("%s runs failed the test" %failed)

['/home/bhavesh.khamesra/Projects/BBH_Injections/IMBH_Studies/GT0577.h5']


0. Waveform Checking - GT0577.h5 



AssertionError: Format entry not found, please check the GT0577.h5 file again

In [ ]:
def read_metadata(h5path_new):
    """Read the metadata from h5file 
    """
    #h5file = h5py.File(h5path_new)
    with h5py.File(h5path_new) as h5file:
        metadata = {}

        for item in h5file.attrs.keys():
            metadata[item] = h5file.attrs[item]


        #Extract the amplitude and phase from h5. If a field is missing, send it to failed directory
        amp, phase = {}, {}

        for key in h5file.keys():
            print key
            if key[:3] =="amp":
                amp_lm =  h5file[key]
                ampkey = amp_lm.keys()
                print amp_lm, ampkey
                #For missing data for some mode in amplitude - reject the waveform
                if len(ampkey)==0:
                    raise ValueError("Data from amplitude - %s missing"%ampkey)
                

                t_val   = amp_lm[ampkey[0]][:]
                amp_val = amp_lm[ampkey[1]][:]
                deg_val = amp_lm[ampkey[2]].value
                err_val = amp_lm[ampkey[3]][:]
                tol_val = amp_lm[ampkey[4]].value

                amp[key] = {"X":t_val, "Y":amp_val, "deg":deg_val, "tol":tol_val, "errors":err_val}

            elif key[:5] =="phase":
                ph_lm = h5file[key]
                phkey = ph_lm.keys()

                #For missing data for some mode in phase  - reject the waveform
                if len(phkey)==0:
                    raise ValueError("Data from phase - %s missing"%phkey)


                t_val   = ph_lm[phkey[0]][:]
                ph_val  = ph_lm[phkey[1]][:]
                deg_val = ph_lm[phkey[2]].value
                err_val = ph_lm[phkey[3]][:]
                tol_val = ph_lm[phkey[4]].value

                phase[key] = {"X":t_val, "Y":ph_val, "deg":deg_val, "tol":tol_val, "errors":err_val}

        #h5_file.close()
        return [metadata, amp, phase]
